## 환경 설정하기

In [ ]:
# 필요한 패키지 설치하기 (한번만 실행)
!pip install ipython-sql

In [1]:
# SQL 확장 기능 로드하기
%load_ext sql

# 메모리 기반 임시 데이터베이스 연결(커널이 재시작 되면 다시 실행 해야함)
%sql sqlite:///:memory:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

#%sql sqlite:///my_test_db1.db ~ 

## OVER 연습하기

- over과 group by 공통점과 차이점

    ~별로 키워드가 나온 문제에 적용해서 푸는데 사용 됌.  
    하지만 group by는 각 행의 정보를 생략하고 집계된 정보만 출력  
    over은 각 행의 정보를 생략하지 않고 집계된 정보를 출력

In [4]:
%%sql

-- 직원 테이블 생성
CREATE TABLE Employees (
    employee_id INT PRIMARY KEY,
    name VARCHAR(50),
    department VARCHAR(50),
    salary INT,
    hire_date DATE
);

-- 직원 데이터 삽입
INSERT INTO Employees (employee_id, name, department, salary, hire_date) VALUES
(101, '김민수', '영업', 50000, '2022-03-01'),
(102, '박지혜', '영업', 55000, '2021-08-15'),
(103, '이진우', '영업', 48000, '2023-01-10'),
(201, '최아영', '개발', 70000, '2020-05-20'),
(202, '정승훈', '개발', 72000, '2019-11-01'),
(203, '윤서아', '개발', 68000, '2022-07-01'),
(301, '강우주', '마케팅', 60000, '2021-01-01'),
(302, '송예나', '마케팅', 62000, '2020-09-10');

-- 판매 테이블 생성
CREATE TABLE Sales (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    amount INT,
    region VARCHAR(50)
);

-- 판매 데이터 삽입
INSERT INTO Sales (order_id, customer_id, order_date, amount, region) VALUES
(1, 10, '2023-01-01', 10000, '서울'),
(2, 11, '2023-01-05', 15000, '부산'),
(3, 10, '2023-01-07', 8000, '서울'),
(4, 12, '2023-01-10', 20000, '경기'),
(5, 11, '2023-01-12', 12000, '부산'),
(6, 13, '2023-01-15', 7000, '서울'),
(7, 10, '2023-02-01', 11000, '서울'),
(8, 14, '2023-02-05', 9000, '대구'),
(9, 15, '2023-02-10', 13000, '경기'),
(10, 12, '2023-02-15', 18000, '경기');

 * sqlite:///:memory:
Done.
8 rows affected.
Done.
10 rows affected.


[]

In [ ]:
%%sql
-- 문제 1: 전체 평균 급여와 직원별 급여 비교하기
select employee_id, name, department, salary, avg(salary) over() as overall_avg_salary from Employees


 * sqlite:///:memory:
Done.


employee_id,name,department,salary,overall_avg_salary
101,김민수,영업,50000,60625.0
102,박지혜,영업,55000,60625.0
103,이진우,영업,48000,60625.0
201,최아영,개발,70000,60625.0
202,정승훈,개발,72000,60625.0
203,윤서아,개발,68000,60625.0
301,강우주,마케팅,60000,60625.0
302,송예나,마케팅,62000,60625.0


In [15]:
%%sql
-- 문제 2: 부서별 평균 급여 계산하기
select employee_id, name, department, salary, avg(salary) over(partition by department) as department_avg_salary from Employees


 * sqlite:///:memory:
Done.


employee_id,name,department,salary,department_avg_salary
201,최아영,개발,70000,70000.0
202,정승훈,개발,72000,70000.0
203,윤서아,개발,68000,70000.0
301,강우주,마케팅,60000,61000.0
302,송예나,마케팅,62000,61000.0
101,김민수,영업,50000,51000.0
102,박지혜,영업,55000,51000.0
103,이진우,영업,48000,51000.0


In [41]:
%%sql
-- 문제 3: 날짜별 누적 판매액 구하기
select order_id, order_date, amount, region, sum(amount) over(order by order_date)  as cumulative_sales from Sales


 * sqlite:///:memory:
Done.


order_id,order_date,amount,region,cumulative_sales
1,2023-01-01,10000,서울,10000
2,2023-01-05,15000,부산,25000
3,2023-01-07,8000,서울,33000
4,2023-01-10,20000,경기,53000
5,2023-01-12,12000,부산,65000
6,2023-01-15,7000,서울,72000
7,2023-02-01,11000,서울,83000
8,2023-02-05,9000,대구,92000
9,2023-02-10,13000,경기,105000
10,2023-02-15,18000,경기,123000


In [47]:
%%sql
-- 문제 4: 부서별 급여 순위 매기기
select employee_id, name, department, salary, rank() over(partition by department order by salary desc) as department_salary_rank from Employees


 * sqlite:///:memory:
Done.


employee_id,name,department,salary,department_salary_rank
202,정승훈,개발,72000,1
201,최아영,개발,70000,2
203,윤서아,개발,68000,3
302,송예나,마케팅,62000,1
301,강우주,마케팅,60000,2
102,박지혜,영업,55000,1
101,김민수,영업,50000,2
103,이진우,영업,48000,3
